In [17]:
from IPython.display import display, HTML
display(HTML("""
<style>
div.container{width:100% !important;}
div.cell.code_cell.rendered{width:98%;}
div.input_prompt{padding:0px;}
div.CodeMirror {font-family:Consolas; font-size:14pt;}
div.CodeMirror-scroll { overflow-x: auto; }
div.text_cell_render.rendered_html{font-size:14pt;}
div.text_cell_render ul li, code{font-size:22pt; line-height:14px;}
div.output {font-size:14pt; font-weight:bold;}
div.input {font-family:Consolas; font-size:14pt;}
div.prompt {min-width:70px;}
div#toc-wrapper{padding-top:120px;}
div.text_cell_render ul li{font-size:14pt;padding:5px;}
table.dataframe{font-size:14px;}
</style>
"""))

# OpenAI Assistants API function calling 기능 실습 (2025년 3월 기준)

본 실습에서는 OpenAI Assistants API의 **함수 호출(Function Calling)** 기능을 사용하여, AI 어시스턴트가 외부 함수를 호출하고 <br>
그 결과를 응답에 활용하는 방법을 단계별로 살펴보겠습니다.<br>
2025년 3월 기준의 OpenAI 공식 API 문서를 바탕으로 최신 함수 호출 기능과 모범 사례를 반영하여 진행합니다. <br>
이 실습은 기존에 ChatGPT API 등을 사용해본 경험이 있고, Assistants API의 새로운 함수 호출 기능에 관심이 있는 개발자를 대상으로 합니다.

## 1. 함수 호출 기능 소개

함수 호출 기능은 모델이 사전에 정의된 함수를 필요에 따라 실행하도록 도와주는 강력한 도구입니다. <br>
이 기능을 통해 어시스턴트는 자체 지식에 없는 **실시간 정보 조회**나 **복잡한 계산 작업** 등을 외부 함수에 위임할 수 있습니다. <br>
예를 들어 사용자가 날씨를 물어보면, 어시스턴트는 날씨 API와 연결된 함수를 호출하여 최신 기온을 가져온 뒤 응답할 수 있습니다. <br>
이처럼 함수 호출을 활용하면:
- **실시간/외부 데이터 활용:** 모델이 최신 정보(날씨, 주가, 뉴스 등)를 함수로부터 받아와 답변에 반영할 수 있습니다.
- **모델 한계 보완:** 수학 계산, 데이터베이스 질의 등 모델이 직접 처리하기 어려운 요청을 외부 로직으로 해결할 수 있습니다.
- **도메인 확장:** 개발자가 정의한 임의의 함수로 모델의 기능을 확장할 수 있어, 특정 분야나 서비스에 특화된 어시스턴트를 구현할 수 있습니다.

OpenAI의 GPT-3.5/4 모델 중 2023년 하반기 이후 출시된 버전들은 이 함수 호출 기능을 지원합니다. <br>
모델에게 함수 목록을 제공하면, 질문 의도에 따라 적절한 함수를 선택해 필요한 인수를 함께 호출 형식으로 응답을 반환합니다. <br>
이제 이러한 함수 호출을 구현하는 방법을 예제로 알아보겠습니다.

## 2. API 키 설정 및 OpenAI 클라이언트 초기화

OpenAI API를 사용하기 위해 먼저 API 키를 준비해야 합니다. <br>
API 키는 OpenAI 계정의 대시보드에서 생성할 수 있으며, 노출되지 않도록 환경 변수 등에 저장하여 사용합니다.<br>
본 예제에서는 **python-dotenv**를 이용해 `.env` 파일에 저장된 키를 로드하고, OpenAI Python SDK의 `OpenAI` 클라이언트를 초기화합니다. <br>
아래 코드에서는 `.env` 파일에서 키를 읽어와 `client = OpenAI()`로 클라이언트를 생성합니다.<br>
(API 키가 올바르게 설정되어 있으면 `OpenAI()` 생성자에서 자동으로 키를 불러옵니다.)


In [2]:
from dotenv import load_dotenv
from decouple import config
from openai import OpenAI
import os
load_dotenv(".env")  # 기본값 ".env" 
client = OpenAI()  

## 3. 외부 함수 정의 및 테스트

이제 함수 호출 기능을 체험하기 위한 예시로 **날씨 정보를 가져오는 함수**를 만들어보겠습니다. 사용자가 "서울 날씨 어때요?"라고 물어보면, <br>
어시스턴트가 이 함수를 호출하여 실시간 날씨 정보를 얻어 답변하도록 해볼 것입니다. 

예를 위해 간단한 `get_weather` 함수를 정의하겠습니다. 이 함수는 위도(latitude)와 경도(longitude)를 받아 해당 위치의 현재 기온을 섭씨로 반환합니다. <br>
구현에는 오픈 메테오(Open-Meteo)라는 공개 기상 API를 사용하여, 주어진 좌표의 현재 기온 데이터를 가져옵니다. <br>
함수는 `requests` 라이브러리를 통해 API를 호출하고, 응답 JSON에서 온도 값을 추출하여 반환합니다. 

함수를 정의한 후, 예시 좌표에 대해 함수를 호출해 제대로 동작하는지 확인해 보겠습니다. <br>
서울의 대략적인 좌표(위도 37.484859, 경도 126.930086)를 입력으로 주었을 때 온도가 잘 반환되는지 출력해 보겠습니다.<br>
```
def get_weather(latitude=37.484859, longitude=126.930086):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
    print(url)
```

In [22]:
# 위도 37.484859, 경도 126.930086
import requests
import json
def get_weather(latitude=37.484642, longitude=126.930063):
    url = f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m"
    print(url)
    response = requests.get(url)
    print(response.text)
    # text -> dict
    # data = json.loads(response.text)
    data = response.json()
    print(data)
    current = data.get('current')
    if current is not None:
        return current.get('temperature_2m')
    else:
        return None
get_weather(37.484859,126.930063)

https://api.open-meteo.com/v1/forecast?latitude=37.484859&longitude=126.930063&current=temperature_2m
{"latitude":37.5,"longitude":126.9375,"generationtime_ms":0.026226043701171875,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":24.0,"current_units":{"time":"iso8601","interval":"seconds","temperature_2m":"°C"},"current":{"time":"2025-12-15T10:15","interval":900,"temperature_2m":2.5}}
{'latitude': 37.5, 'longitude': 126.9375, 'generationtime_ms': 0.026226043701171875, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 24.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2025-12-15T10:15', 'interval': 900, 'temperature_2m': 2.5}}


2.5

## 4. 함수 툴 설정 및 모델 응답 생성

이제 OpenAI API에 우리가 정의한 함수를 **툴(tool)**로 등록하고, 모델이 이 함수를 호출하도록 유도해 보겠습니다. 
이를 위해 먼저 `tools`라는 리스트에 함수 정보를 구성합니다. 
각 함수 툴에는 이름(name), 설명(description), 그리고 매개변수(parameters) 스키마를 지정해야 합니다.
아래 코드에서는 `type`: "function"으로 함수를 정의하고, `name`: "get_weather", 
              `description`: "저장된 좌표의 현재 온도를 섭씨 단위로 구합니다"와 같이 설정했습니다.
          또한 `parameters` 필드에 함수가 요구하는 인자들의 JSON 스키마(여기서는 latitude와 longitude 숫자형, 둘 다 필수)를 명시했습니다. 

그 다음 어시스턴트에게 보낼 `messages`를 준비합니다. 사용자의 질문으로 "오늘 서울 날씨 어때요?"라는 메시지를 추가하겠습니다. 
이제 이 사용자 메시지와 함께 <b> `client.chat.completions.create` </b>를 호출하여, 모델의 응답을 받아보겠습니다. 
`tools` 파라미터에 앞서 정의한 함수 정보를 포함시켰으므로, 모델은 답변 과정에서 이 함수를 호출할 수 있게 됩니다.
모델이 함수 호출을 결정하면, 응답으로 일반 텍스트 대신 **함수 호출 요청**을 반환하게 됩니다.
즉, 어시스턴트는 직접 답을 주는 대신 `get_weather` 함수를 특정 인수로 호출하라는 정보를 주게 됩니다. 
아래 코드에서는 ChatCompletion API를 호출하여 이러한 과정을 실행하고, 결과를 `completion` 변수에 저장합니다.


In [23]:
messages = [{"role":"user", "content":"오늘 제주 날씨 어때요?"}]
tools = [{
    "type":"function",
    "function":{
        "name":"get_weather",
        "description":"저장된 좌표의 현재 온도를 섭씨 단위로 구합니다",
        "parameters":{
            "type":"object",
            "properties":{
                "latitude":{"type":"number"}, # 위도
                "longitude":{"type":"number"} # 경도
            }, # properties 끝
            "required":["latitude", "longitude"],
            "additionalProperties" :False # 지정된 properties 이외에는 추가적인 키를 허용 안 함
        } # parameters 끝
    }, # function 끝
    "strict":True   # 함수 호출 시 정의된 JSON Schema를 100% 준수하도록 강제하는 옵션
}]
completion = client.chat.completions.create(
    model="gpt-4.1-nano",
    messages=messages,
    tools=tools,
    parallel_tool_calls=False # 병렬처리하지 않고 하나만 받음
)

## 5. 함수 호출 결과 처리 및 최종 답변
```
모델이 함수 호출 요청을 반환했다면, 이제 개발자 측에서 해당 함수를 실제로 실행하고 그 결과를 어시스턴트에게 전달해야 합니다. 
우선 `completion.choices[0].message.tool_calls`를 통해 모델이 요청한 함수 호출 정보를 확인해보겠습니다. 
여기에는 호출하려는 함수 이름과 전달된 인자들이 포함되어 있습니다. 우리 예시의 경우 모델은 `get_weather` 함수를 호출하도록 응답했을 것이며, 
서울의 위도와 경도에 해당하는 값을 인자로 제공했을 것입니다. 
모델이 반환한 함수 호출 정보 (tool_call)를 이용해 실제 `get_weather` 함수를 실행하고, 그 결과(현재 기온)를 얻어보겠습니다. 
그런 다음 이 결과를 다시 대화의 맥락에 추가하여, 어시스턴트가 최종 답변을 생성할 수 있도록 합니다. 

먼저, 모델 응답에 담긴 `tool_calls` 목록을 확인하고, 필요한 인자를 추출해 함수를 호출해보겠습니다.
```

In [16]:
completion #제주 날씨어때에 대한 응답

ChatCompletion(id='chatcmpl-CmrnHL5LPogyppca5amLSYPGqnWSf', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_5rqsvT2pIJ7NM7sr6mSP9F5j', function=Function(arguments='{"latitude":33.38,"longitude":126.53}', name='get_weather'), type='function')]))], created=1765762563, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_7f8eb7d1f9', usage=CompletionUsage(completion_tokens=22, prompt_tokens=66, total_tokens=88, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [6]:
completion # 서울 날씨 어때에 대한 응답

ChatCompletion(id='chatcmpl-CmrYb3xGAUdUyBApaSMU9d5588VsD', choices=[Choice(finish_reason='tool_calls', index=0, logprobs=None, message=ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_uLkLUybCEQ6FQOvY05OvKFaY', function=Function(arguments='{"latitude":37.5665,"longitude":126.978}', name='get_weather'), type='function')]))], created=1765761653, model='gpt-4.1-nano-2025-04-14', object='chat.completion', service_tier='default', system_fingerprint='fp_7f8eb7d1f9', usage=CompletionUsage(completion_tokens=23, prompt_tokens=66, total_tokens=89, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))

In [7]:
# llm이 자연어분석하여 "서울"의 좌표를 내부적으로 매핑
completion.choices[0].message.tool_calls

[ChatCompletionMessageFunctionToolCall(id='call_uLkLUybCEQ6FQOvY05OvKFaY', function=Function(arguments='{"latitude":37.5665,"longitude":126.978}', name='get_weather'), type='function')]

In [29]:
import json
for tool_call in completion.choices[0].message.tool_calls:
    name = tool_call.function.name
    arg  = json.loads(tool_call.function.arguments)
    print(name, arg, type(arg))
    latitude = arg.get('latitude')
    longitude = arg.get('longitude')
    print(name, latitude, longitude)
    if name == 'get_weather':
        result = get_weather(latitude, longitude)
        print(result)

get_weather {'latitude': 33.3775, 'longitude': 126.5219} <class 'dict'>
get_weather 33.3775 126.5219
https://api.open-meteo.com/v1/forecast?latitude=33.3775&longitude=126.5219&current=temperature_2m
{"latitude":33.4,"longitude":126.5,"generationtime_ms":0.022172927856445312,"utc_offset_seconds":0,"timezone":"GMT","timezone_abbreviation":"GMT","elevation":1612.0,"current_units":{"time":"iso8601","interval":"seconds","temperature_2m":"°C"},"current":{"time":"2025-12-15T11:00","interval":900,"temperature_2m":1.1}}
{'latitude': 33.4, 'longitude': 126.5, 'generationtime_ms': 0.022172927856445312, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 1612.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2025-12-15T11:00', 'interval': 900, 'temperature_2m': 1.1}}
1.1


In [25]:
print(messages)

[{'role': 'user', 'content': '오늘 제주 날씨 어때요?'}]


In [26]:
tool_call

ChatCompletionMessageFunctionToolCall(id='call_ANGPCgzBlwg9pmaK8gMQRmLJ', function=Function(arguments='{"latitude":33.3775,"longitude":126.5219}', name='get_weather'), type='function')

In [27]:
# messages에 tools를 추가
messages.append(completion.choices[0].message)
messages.append({
    "role":"tool",
    "tool_call_id":tool_call.id,
    "content":str(result) # content는 스트링이나 리스트로 
})

In [28]:
messages

[{'role': 'user', 'content': '오늘 제주 날씨 어때요?'},
 ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageFunctionToolCall(id='call_ANGPCgzBlwg9pmaK8gMQRmLJ', function=Function(arguments='{"latitude":33.3775,"longitude":126.5219}', name='get_weather'), type='function')]),
 {'role': 'tool',
  'tool_call_id': 'call_ANGPCgzBlwg9pmaK8gMQRmLJ',
  'content': '0.9'}]

```
함수 실행 결과를 어시스턴트 대화에 추가해 보겠습니다. `messages.append(...)`를 사용하여, 먼저 어시스턴트의 함수 호출 메시지를 대화 내역에 넣고,
이어서 'tool' 역할의 메시지를 추가합니다. 이 'tool' 메시지에는 함수 호출의 `id`와 실행 결과를 문자열로 담았습니다. 
이렇게 하면 OpenAI API는 해당 함수 호출에 대한 결과를 받았다고 인식하게 됩니다.
```

In [13]:
response = client.chat.completions.create(
    model = "gpt-4.1-nano",
    messages=messages,
    tools=tools
)

In [14]:
response.choices[0].message.content

'오늘 서울의 현재 온도는 약 -1.3도입니다. 춥기 때문에 따뜻하게 입고 외출하시기 바랍니다.'